In [11]:
from pony.orm import db_session, select, commit
from be import db
from be.models import User, Course, Test, Question, Option, Topic, UserAnswer, UserCourseSelection
from be.background_tasks import add_questions_to_test
import json

## Load seed data into db

In [2]:
db.bind(
    provider = 'mysql',
    host = 'localhost',
    user = 'its_admin',
    password = 'test123',
    db ='its-mysql'
)
db.generate_mapping(create_tables=True)

In [3]:
with open('seed/courses_list.json', 'rb') as seed_data:
    seed_data = json.load(seed_data)
    with db_session:
        for course in seed_data.keys():
            topics = []
            for t in seed_data[course]:
                topics.append(Topic(topic=t))
            Course(name=course, topics=topics)

TransactionIntegrityError: Object Topic['Algebra'] cannot be stored in the database. IntegrityError: 1062 Duplicate entry 'Algebra' for key 'topic.PRIMARY'

In [4]:
with db_session:
    if not User.get(username="admin"):
        User(username="admin",password="pbkdf2:sha256:600000$XrquGmU5oGJhnxvp$a6eee924e0e7fb1fc53ac91517a8010527e77801da4e8b4d1f8c77bb95b2fcaf")

In [5]:
with db_session:
    user = User.get(username="admin")
    courses = Course.select()
    for course in courses:
        if course not in user.enrolled_courses:
            user_course = UserCourseSelection(user=user, course=course)
            user.enrolled_courses.add(user_course)

In [ ]:
with db_session:
    user = User.get(username="admin")
    for ucs in user.enrolled_courses:
        for x in range(100):
            course = ucs.course
            topics = course.topics
            new_test = Test(user_course_selection=ucs, topics=topics)
            commit()
            add_questions_to_test(new_test.id)